# Prevendo novos dados

> Jean Carlos, Ph.D.

In [1]:
# importando pacotes
import joblib
import pandas as pd
import numpy as np

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
# Lendo os novos dados
new_data = pd.read_parquet('/content/drive/MyDrive/case_rentcars/data/processed/data_test.parquet')
print(new_data.shape)

(4833292, 30)


In [7]:
# Para carregar o modelo salvo
model_ = joblib.load('/content/drive/MyDrive/case_rentcars/models/modelo_treinado.pkl')

# Caminho onde as features foram salvas
path_to_load = '/content/drive/MyDrive/case_rentcars/models/selected_features.txt'

# Carregar as features salvas
features = pd.read_csv(path_to_load, header=None).squeeze("columns").tolist()

# Definir o atributo features no modelo carregado
model_.features = features

# Supondo que você tenha um DataFrame new_data com as colunas 'order_id' e 'product_id'
new_data['pred'] = model_.predict(new_data[model_.features])
test_data = new_data[['order_id','product_id','pred']]

# Filtrar produtos previstos como 1
predicted_products = test_data[test_data['pred'] == 1]

# Agrupar por 'order_id' e criar strings de IDs de produtos
submission = predicted_products.groupby('order_id')['product_id'].apply(lambda x: ' '.join(x.astype(str))).reset_index()

# Renomear colunas conforme necessário
submission.columns = ['order_id', 'products']
submission.head()

,order_id,products
0,17,21709 21903 47766
1,34,651 2361 2596 5134 6317 7035 7131 7559 7948 93...
2,137,2078 2326 11512 16797 18465 21903 23794 24184 ...
3,182,1244 1757 2078 2295 2480 3397 4344 4421 5479 6...
4,257,2063 4605 8277 10621 11140 11782 12341 12456 1...


In [8]:
# Salvar o DataFrame de submissão
submission.to_csv('/content/drive/MyDrive/case_rentcars/models/submission.csv', index=False)
